<a href="https://colab.research.google.com/github/deedee-ke/practice/blob/main/Chapter_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:

  def _init_(self, product, quantity, price):
    self.product = product
    self.quantity = quantity
    self.price = price

  def total(self):
    return self.price * self.quantity

class Order:

  def _init_(self, customer, cart, promotion=None):
    self.customer = customer
    self.cart = list(cart)
    self.promotion = promotion

  def total(self):
    if not hasattr(self, '__total'):
      self.__total = sum(item.total() for item in self.cart)
    return self.__total

  def due(self):
    if self.promotion is None:
      discount = 0
    else:
      discount = self.promotion.discount(self)
    return self.total() - discount

  def _repr_(self):
    fmt = '<Order total: {:.2f} due: {:.2f}>'
    return fmt.format(self.total(), self.due())

class Promotion(ABC):

  @abstractmethod
  def discount(self, order):
    """Retrun discount as a positive dollar amount"""

class FidelityPromo(Promotion):
  """5% discount for customers with 1000 or more fidelity points"""

  def discount(self, order):
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

class BuldItemPromo(Promotion):
  """10% discount for each LineItem with 20 or more units"""

  def discount(self, order):
    discount = 0
    for item in order.cart:
      if item.quantity >= 20:
        discount += item.total() * .1
    return discount

class LargeOrderPromo(Promotion):
  """7% discount for orders with 10 or more distinct items"""

  def discount(self, order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
      return order.total() * .07
    return 0


In [2]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]
Order(joe, cart, FidelityPromo())

TypeError: ignored